### This notebook investigates the role that dead-end metabolite Nitrous Oxide plays in the E. coli metabolism
A small purpose is to activate RXN0-2721, which is the reaction that produces nitrous oxide in cytosol.
- [ ] remove oxygen exchange + add reaction that breaks nitrous oxide into nitrogen and oxygen
- [ ] find info on dinitrification of nitrous oxide in literature
- [ ] consider kinetic rates for Nitrogen (N) related reactions

In [1]:
import numpy as np
import ast
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
import networkx as nx
import plotly.express as px

from scipy.special import logsumexp

%matplotlib inline
# sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.library.schema import numpy_schema, bulk_name_to_idx, listener_schema, counts
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

In [4]:
# load basal new model
time = '400'
date = '2025-01-30'
experiment = 'NEW_updated_bad_rxn'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry
reaction_names = metabolism.reaction_names
kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
binary_kinetic_idx = metabolism.binary_kinetic_idx
metabolites = metabolism.metabolite_names

homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).loc[24, :]
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).loc[24, :]
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :]

In [7]:
uptake = metabolism.allowed_exchange_uptake
uptake = set(uptake)

addition = set([
    # "NITROUS-OXIDE[e]",
    # "NITRATE[p]",
])
removals = set([
    "OXYGEN-MOLECULE[p]",
])


uptake = uptake | addition
uptake = uptake - removals

In [2]:
# #TODO: Add transport of and reduction of NITROUS-OXIDE[p] to [c] and in [c]
# print(stoichiometry.shape)
# new_reaction_names = metabolism.reaction_names.copy()
# new_reaction_names.append('TEMP-Dinitrification')
# 
# new_reaction = np.zeros((stoichiometry.shape[0], 1))
# new_reaction[metabolites.index('NITROUS-OXIDE[c]'), 0] = -2
# new_reaction[metabolites.index('OXYGEN-MOLECULE[c]'), 0] = 1
# new_reaction[metabolites.index('NITROGEN-MOLECULE[c]'), 0] = 1
# 
# 
# new_stoichiometry = np.concatenate((stoichiometry, new_reaction), axis=1)
# new_stoichiometry.shape

NameError: name 'stoichiometry' is not defined

In [3]:
model = NetworkFlowModel(
            stoich_arr=new_stoichiometry,
            metabolites=metabolism.metabolite_names,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
model.set_up_exchanges(exchanges=metabolism.exchange_molecules, uptakes=uptake)
solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_count * metabolism.counts_to_molar.asNumber(), # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            binary_kinetic_idx=binary_kinetic_idx,
            force_flow_idx=None,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.   
            solver=cp.GLOP)


NameError: name 'new_stoichiometry' is not defined

In [6]:
metabolites.index('NITROGEN-MOLECULE[c]')

ValueError: 'NITROGEN-MOLECULE[c]' is not in list